In [9]:
!pip install opencv-python mediapipe numpy


In [15]:
import cv2
import mediapipe as mp
import numpy as np
import time

class GestureController:
    def __init__(self):
        # Inicializar componentes de MediaPipe para detección de manos
        self.mp_hands = mp.solutions.hands
        self.hands = self.mp_hands.Hands(
            static_image_mode=False,
            max_num_hands=2,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5
        )
        self.mp_drawing = mp.solutions.drawing_utils
        self.mp_drawing_styles = mp.solutions.drawing_styles
        
        # Variables para el control de interacción
        self.bg_color = (0, 0, 0)  # Color de fondo inicial (negro)
        self.object_position = [320, 240]  # Posición inicial del objeto [x, y]
        self.scene = 0  # Escena actual (0, 1, 2)
        self.scene_names = ["Mover Objeto", "Cambiar Color", "Dibujar"]
        self.prev_hand_gesture = None
        self.gesture_cooldown = 0  # Cooldown para evitar cambios rápidos
        
        # Variables para dibujo
        self.drawing_points = []
        self.is_drawing = False
        self.draw_color = (0, 255, 0)  # Color verde para dibujar
    
    def process_frame(self, frame):
        # Convertir la imagen a RGB (MediaPipe requiere RGB)
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        h, w, _ = frame.shape
        
        # Procesar la imagen para detectar manos
        results = self.hands.process(image_rgb)
        
        # Crear un marco transparente para dibujar sobre la imagen original
        overlay = frame.copy()
        
        # Dibujar el fondo según la escena actual
        if self.scene == 1:  # En la escena de cambio de color
            cv2.rectangle(overlay, (0, 0), (w, h), self.bg_color, -1)
        
        # Verificar si se detectaron manos
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Dibujar los puntos de referencia de la mano
                self.mp_drawing.draw_landmarks(
                    overlay, 
                    hand_landmarks, 
                    self.mp_hands.HAND_CONNECTIONS,
                    self.mp_drawing_styles.get_default_hand_landmarks_style(),
                    self.mp_drawing_styles.get_default_hand_connections_style()
                )
                
                # Procesar gestos según la escena
                if self.scene == 0:  # Escena: Mover Objeto
                    self.handle_move_object(hand_landmarks, w, h)
                elif self.scene == 1:  # Escena: Cambiar Color
                    self.handle_change_color(hand_landmarks)
                elif self.scene == 2:  # Escena: Dibujar
                    self.handle_drawing(hand_landmarks, overlay, w, h)
                
                # Detectar gesto para cambiar de escena (palma abierta)
                if time.time() > self.gesture_cooldown:
                    current_gesture = self.detect_open_palm(hand_landmarks)
                    
                    # Si detectamos palma abierta y antes no estaba, cambiamos escena
                    if current_gesture == "open_palm" and self.prev_hand_gesture != "open_palm":
                        self.scene = (self.scene + 1) % len(self.scene_names)
                        self.gesture_cooldown = time.time() + 1.5  # 1.5 segundos de cooldown
                        self.drawing_points = []  # Limpiar puntos al cambiar escena
                    
                    self.prev_hand_gesture = current_gesture
        
        # Mezclar la imagen original con la capa de overlay
        frame = cv2.addWeighted(overlay, 0.7, frame, 0.3, 0)
        
        # Dibujar elementos específicos de la escena actual
        if self.scene == 0:  # Escena: Mover Objeto
            # Dibujar el objeto controlado
            cv2.circle(frame, (self.object_position[0], self.object_position[1]), 30, (0, 0, 255), -1)
        elif self.scene == 2:  # Escena: Dibujar
            # Dibujar los trazos del usuario
            if len(self.drawing_points) > 1:
                for i in range(1, len(self.drawing_points)):
                    if self.drawing_points[i-1] is not None and self.drawing_points[i] is not None:
                        cv2.line(frame, self.drawing_points[i-1], self.drawing_points[i], self.draw_color, 5)
        
        # Mostrar la escena actual en la pantalla
        cv2.putText(frame, f"Escena: {self.scene_names[self.scene]}", (10, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        
        if self.scene == 0:
            cv2.putText(frame, "Mueve el indice para controlar el circulo", (10, 70), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        elif self.scene == 1:
            cv2.putText(frame, "Abre/cierra la mano para cambiar color", (10, 70), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        elif self.scene == 2:
            cv2.putText(frame, "Indice+pulgar juntos para dibujar", (10, 70), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        
        cv2.putText(frame, "Muestra la palma abierta para cambiar de escena", (10, h-30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            
        return frame
    
    def handle_move_object(self, hand_landmarks, width, height):
        # Usar la punta del dedo índice para mover el objeto
        index_tip = hand_landmarks.landmark[self.mp_hands.HandLandmark.INDEX_FINGER_TIP]
        x, y = int(index_tip.x * width), int(index_tip.y * height)
        
        # Actualizar la posición del objeto
        self.object_position = [x, y]
    
    def handle_change_color(self, hand_landmarks):
        # Contar dedos extendidos para cambiar color
        fingers_extended = self.count_fingers_extended(hand_landmarks)
        
        # Cambiar color según el número de dedos extendidos
        if fingers_extended == 0:
            self.bg_color = (0, 0, 0)  # Negro
        elif fingers_extended == 1:
            self.bg_color = (0, 0, 255)  # Rojo (BGR)
        elif fingers_extended == 2:
            self.bg_color = (0, 255, 0)  # Verde
        elif fingers_extended == 3:
            self.bg_color = (255, 0, 0)  # Azul
        elif fingers_extended == 4:
            self.bg_color = (0, 255, 255)  # Amarillo
        elif fingers_extended == 5:
            self.bg_color = (255, 0, 255)  # Magenta
    
    def handle_drawing(self, hand_landmarks, frame, width, height):
        # Obtener coordenadas de las puntas de los dedos índice y pulgar
        index_tip = hand_landmarks.landmark[self.mp_hands.HandLandmark.INDEX_FINGER_TIP]
        thumb_tip = hand_landmarks.landmark[self.mp_hands.HandLandmark.THUMB_TIP]
        
        # Convertir coordenadas normalizadas a píxeles
        index_x, index_y = int(index_tip.x * width), int(index_tip.y * height)
        thumb_x, thumb_y = int(thumb_tip.x * width), int(thumb_tip.y * height)
        
        # Calcular la distancia entre el pulgar y el índice
        distance = ((index_x - thumb_x) ** 2 + (index_y - thumb_y) ** 2) ** 0.5
        
        # Si los dedos están cerca, estamos en modo dibujo
        if distance < 40:  # Umbral de distancia para considerar "pinza"
            if not self.is_drawing:
                self.is_drawing = True
                self.drawing_points.append(None)  # Marca un nuevo trazo
            else:
                self.drawing_points.append((index_x, index_y))
        else:
            self.is_drawing = False
    
    def count_fingers_extended(self, hand_landmarks):
        # Puntos de referencia de las puntas de los dedos
        finger_tips = [
            self.mp_hands.HandLandmark.THUMB_TIP,
            self.mp_hands.HandLandmark.INDEX_FINGER_TIP,
            self.mp_hands.HandLandmark.MIDDLE_FINGER_TIP,
            self.mp_hands.HandLandmark.RING_FINGER_TIP,
            self.mp_hands.HandLandmark.PINKY_TIP
        ]
        # Puntos de referencia de las articulaciones intermedias (para verificar si el dedo está extendido)
        finger_pips = [
            self.mp_hands.HandLandmark.THUMB_IP,  # Para el pulgar usamos IP en lugar de PIP
            self.mp_hands.HandLandmark.INDEX_FINGER_PIP,
            self.mp_hands.HandLandmark.MIDDLE_FINGER_PIP,
            self.mp_hands.HandLandmark.RING_FINGER_PIP,
            self.mp_hands.HandLandmark.PINKY_PIP
        ]
        # Punto de referencia de la muñeca
        wrist = hand_landmarks.landmark[self.mp_hands.HandLandmark.WRIST]
        
        count = 0
        # Para el pulgar, el criterio es diferente (miramos si está lejos de la palma)
        thumb_tip = hand_landmarks.landmark[finger_tips[0]]
        thumb_base = hand_landmarks.landmark[self.mp_hands.HandLandmark.THUMB_CMC]
        if thumb_tip.x < thumb_base.x:  # Simplificado para mano derecha
            count += 1
        
        # Para los demás dedos
        for i in range(1, 5):
            tip = hand_landmarks.landmark[finger_tips[i]]
            pip = hand_landmarks.landmark[finger_pips[i]]
            
            # Si la punta del dedo está más arriba que la articulación, el dedo está extendido
            if tip.y < pip.y:
                count += 1
                
        return count
    
    def detect_open_palm(self, hand_landmarks):
        # Contar dedos extendidos
        fingers = self.count_fingers_extended(hand_landmarks)
        
        # Si todos los dedos están extendidos, es palma abierta
        if fingers >= 4:
            return "open_palm"
        return "other"

def main():
    # Inicializar la cámara
    cap = cv2.VideoCapture(1)
    
    # Inicializar el controlador de gestos
    gesture_controller = GestureController()
    
    while cap.isOpened():
        # Leer un frame de la cámara
        success, frame = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            continue
        
        # Voltear la imagen horizontalmente para una vista tipo "espejo"
        frame = cv2.flip(frame, 1)
        
        # Procesar el frame con nuestro controlador de gestos
        output_frame = gesture_controller.process_frame(frame)
        
        # Mostrar la imagen resultante
        cv2.imshow('MediaPipe Hands Gesture Control', output_frame)
        
        # Salir si se presiona ESC
        if cv2.waitKey(5) & 0xFF == 27:
            break
    
    # Liberar recursos
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

In [13]:
import cv2

cap = cv2.VideoCapture(1)

if not cap.isOpened():
    print("No se pudo abrir la cámara.")
else:
    print("Cámara conectada correctamente.")

cap.release()


Cámara conectada correctamente.
